# Designing an Airdrop - A Case Study with Paraswap
> All's fair in Love & Airdrops

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [airdrops, paraswap]
- hide: false  


In [16]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#import numpy as np
%matplotlib inline
%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))
%R suppressMessages(install.packages("ineq"))
%R suppressMessages(library(ineq))
#print(px.colors.qualitative.D3)


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/ineq_0.2-13.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 21946 bytes (21 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =

'ineq','jsonlite','lubridate',...,'datasets','methods','base'


In [17]:
#hide
%%R

#This code block does the on-chain import from flipside for the psp token distribution charts

#Grab eth query from Flipside
eth1 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/14044a0c-34ca-4b1f-af6a-fba1f1f432e3/data/latest', simplifyDataFrame = TRUE)
eth2 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/11672489-ed37-4a53-996b-272d24e0ffbc/data/latest', simplifyDataFrame = TRUE)

df_eth = eth1 %>% bind_rows(eth2)

#fix the column names
names(df_eth)<-tolower(names(df_eth))
df_eth <- df_eth %>% rename(address = origin_address,
                    eth_sum_usd = sum_usd,
                    eth_tx_count = tx_count)

#grab polygon data from Flipside
poly1 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/6deea9f0-86d8-4088-a686-62b752f3b89d/data/latest', simplifyDataFrame = TRUE)
poly2 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/e3a19d33-a058-49c7-81b9-8a0d510df4be/data/latest', simplifyDataFrame = TRUE)
poly3 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/e0ecb2b9-b2e6-4b78-8b36-070b7f1a3fc9/data/latest', simplifyDataFrame = TRUE)
poly4 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/7501f8a0-2161-4f2d-a4d4-0ebf6f8ef4af/data/latest', simplifyDataFrame = TRUE)
poly5 = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/c0dcaf3b-2dad-498f-bfe3-bb0f53944b4f/data/latest', simplifyDataFrame = TRUE)

df_poly <- poly1 %>%
  bind_rows(poly2) %>%
  bind_rows(poly3) %>%
  bind_rows(poly4) %>%
  bind_rows(poly5)

#fix the column names
names(df_poly)<-tolower(names(df_poly))
df_poly <- df_poly %>% rename(address = origin_address,
                    poly_sum_usd = sum_usd,
                    poly_tx_count = tx_count)


#Join the tables together & fill the nas with zero
df <- df_eth %>%
  full_join(df_poly) %>%
  replace_na(list(eth_sum_usd = 0,
                  eth_tx_count = 0,
                  poly_sum_usd = 0,
                  poly_tx_count = 0)) %>%
  mutate(sum_usd = eth_sum_usd + poly_sum_usd,
         tx_count = eth_tx_count + poly_tx_count)
  
#grab the airdrop list from github
airdrops = fromJSON('https://raw.githubusercontent.com/paraswap/paraswap-rewards-snapshot/master/data/airdrop-users.json', simplifyDataFrame = TRUE)

#fix the token amount
airdrops$earnings <- parse_double(airdrops$earnings)
airdrops <- airdrops %>% mutate(earnings = earnings / 1e18)

#join the airdrops into the user table 
df <- df %>%
  left_join(airdrops, by = 'address') %>%
  mutate(earnings = replace_na(earnings, 0),
         got_airdrop = if_else(earnings > 0, 1, 0)) 

#create bins for tx_size
df <- df %>%
  mutate(eth_tx_bucket = case_when(
    eth_tx_count == 0 ~ "0",
    eth_tx_count == 1 ~ "1",
    eth_tx_count == 2 ~ "2",
    eth_tx_count == 3 ~ "3",
    eth_tx_count == 4 ~ "4",
    eth_tx_count == 5 ~ "5",
    eth_tx_count == 6 ~ "6",
    eth_tx_count == 7 ~ "7",
    eth_tx_count == 8 ~ "8",
    eth_tx_count == 9 ~ "9",
    eth_tx_count >= 100 ~ "100+",
    eth_tx_count >= 50 ~ "50-100",
    eth_tx_count >= 20 ~ "20-50",
    eth_tx_count >= 10 ~ "10-20"
  )) %>%
  mutate(poly_tx_bucket = case_when(
    poly_tx_count == 0 ~ "0",
    poly_tx_count == 1 ~ "1",
    poly_tx_count == 2 ~ "2",
    poly_tx_count == 3 ~ "3",
    poly_tx_count == 4 ~ "4",
    poly_tx_count == 5 ~ "5",
    poly_tx_count == 6 ~ "6",
    poly_tx_count == 7 ~ "7",
    poly_tx_count == 8 ~ "8",
    poly_tx_count == 9 ~ "9",
    poly_tx_count >= 100 ~ "100+",
    poly_tx_count >= 50 ~ "50-100",
    poly_tx_count >= 20 ~ "20-50",
    poly_tx_count >= 10 ~ "10-20"
  )) %>%
  mutate(tx_bucket = case_when(
    tx_count == 0 ~ "0",
    tx_count == 1 ~ "1",
    tx_count == 2 ~ "2",
    tx_count == 3 ~ "3",
    tx_count == 4 ~ "4",
    tx_count == 5 ~ "5",
    tx_count == 6 ~ "6",
    tx_count == 7 ~ "7",
    tx_count == 8 ~ "8",
    tx_count == 9 ~ "9",
    tx_count >= 100 ~ "100+",
    tx_count >= 50 ~ "50-100",
    tx_count >= 20 ~ "20-50",
    tx_count >= 10 ~ "10-20"
  )) %>%
  mutate(eth_usd_bucket = case_when(
    eth_sum_usd == 0 ~ "0",
    eth_sum_usd < 500 ~ "0-500",
    eth_sum_usd < 1000 ~ "500-1k",
    eth_sum_usd < 2000 ~ "1k-2k",
    eth_sum_usd < 5000 ~ "2k-5k",
    eth_sum_usd < 10000 ~ "5k-10k",
    eth_sum_usd < 100000 ~ "10k-100k",
    eth_sum_usd < 1000000 ~ "100k-1m",
    eth_sum_usd >= 1000000 ~ "1m+",
  ))

df$eth_tx_bucket = parse_factor(df$eth_tx_bucket, ordered = TRUE, levels = c("0","1","2","3","4","5","6","7","8","9","10-20","20-50","50-100","100+"))

df$poly_tx_bucket = parse_factor(df$poly_tx_bucket, ordered = TRUE, levels = c("0","1","2","3","4","5","6","7","8","9","10-20","20-50","50-100","100+"))

df$tx_bucket = parse_factor(df$tx_bucket, ordered = TRUE, levels = c("0","1","2","3","4","5","6","7","8","9","10-20","20-50","50-100","100+"))

df$eth_usd_bucket = parse_factor(df$eth_usd_bucket, ordered = TRUE, levels = c("0","0-500","500-1k","1k-2k","2k-5k","5k-10k","10k-100k","100k-1m","1m+"))

#group & sort for charts
eth_by_tx_bucket <- df %>%
  filter(eth_tx_count > 0) %>%
  group_by(eth_tx_bucket) %>%
  summarise(num = n(),
            airdrops = sum(got_airdrop),
            percent = airdrops / num * 100) %>%
  arrange(eth_tx_bucket) %>% 
  ungroup() %>%
  mutate(average = sum(airdrops) / sum(num) * 100)

poly_by_tx_bucket <- df %>%
  filter(poly_tx_count > 0) %>%
  group_by(poly_tx_bucket) %>%
  summarise(num = n(),
            airdrops = sum(got_airdrop),
            percent = airdrops / num * 100) %>%
  arrange(poly_tx_bucket) %>% 
  ungroup() %>%
  mutate(average = sum(airdrops) / sum(num) * 100)

by_tx_bucket <- df %>%
  group_by(tx_bucket) %>%
  summarise(num = n(),
            airdrops = sum(got_airdrop),
            percent = airdrops / num * 100) %>%
  arrange(tx_bucket) %>% 
  ungroup() %>%
  mutate(average = sum(airdrops) / sum(num) * 100)

eth_value_bucket <- df %>%
  filter(eth_tx_count > 0) %>%
  group_by(eth_usd_bucket) %>%
  summarise(num = n(),
            airdrops = sum(got_airdrop),
            percent = airdrops / num * 100) %>%
  arrange(eth_usd_bucket) %>% 
  ungroup() %>%
  mutate(average = sum(airdrops) / sum(num) * 100)

  eth_users <- df %>%
  mutate(eth_user = if_else(eth_tx_count > 0, 1, 0)) %>%
  summarise("Network" = "Ethereum",
            "Users" = sum(eth_user),
            "Transactions" = sum(eth_tx_count))

poly_users <- df %>%
  mutate(poly_user = if_else(poly_tx_count > 0, 1, 0)) %>%
  summarise("Network" = "Polygon",
            "Users" = sum(poly_user),
            "Transactions" = sum(poly_tx_count))

all_users <- df %>%
  summarise("Network" = "Total",
            "Users" = n(),
            "Transactions" = sum(tx_count))


Joining, by = "address"


In [18]:
#hide
%%R

#This block grabs the other token airdrop data & transforms into a vector of individual allocations
# this may take a while to run - UNI is a very large dataset due to inclusions of proofs in the source data

#grab the ENS airdrop list from github (use stream_in due to newline delimited JSON)
ens <- stream_in(file('https://raw.githubusercontent.com/ensdomains/governance/master/airdrop.json'))

#fix token decimals
ens$past_tokens <- parse_double(ens$past_tokens) / 1e18
ens$future_tokens <- parse_double(ens$future_tokens) / 1e18

#get the df of token amounts
ens <- ens %>%
  mutate(tokens = (past_tokens + future_tokens)) %>%
  select(tokens)


#get the non-zero psp allocations
df_nonzero <- df %>% filter(earnings > 0) %>% select(earnings)

####################################################
# the following code grabs the UNI airdrop data.  This is a huge dataset (1.2Gb) which can cause it to fail during download.
# I've instead posted the results to GH as a csv.  Uncomment below to use the original code
#grab the UNI airdrop list from mrkl.uniswap.org
#uni <- fromJSON('https://mrkl.uniswap.org/', simplifyDataFrame = TRUE, flatten = TRUE)

#uni_claims_list <- uni$claims

#define the function to extract the amount field
#return_amount <- function(x) x$amount

#purrr:map to get a list of the amounts
#uni_amounts <- map(uni_claims_list, return_amount)

#transpose and convert to numbers
#uni_claims <- uni_amounts %>%
#  t() %>%
#  as.double() / 1e18

#rm(uni)
#rm(uni_claims_list)
####################################################

#grab the processed copy of UNI from my github
uni_claims <- read_csv('https://raw.githubusercontent.com/scottincrypto/analytics/master/data/uni_claims.csv')$x

#grab the RBN airdrop list from github
rbn <- fromJSON('https://raw.githubusercontent.com/ribbon-finance/governance/main/airdrop-data/airdrop.json', simplifyDataFrame = TRUE)

#transpose the data
rbn <- rbn %>%
  as_tibble() %>%
  t()

#fix up the hex formatting & convert to a number
rbn_hex <- paste("0x", rbn, sep="")
rbn_claims <- rbn_hex %>% as.double() / 1e18



R[write to console]: opening url input connection.



 Imported 137689 records. Simplifying...


R[write to console]: closing url input connection.

R[write to console]: New names:
* `` -> ...1



Rows: 252803 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (2): ...1, x

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [26]:
#hide
%%R
#This section does the Lorenz Curve & inequality index calcs on the token distribution data

#Do the Lorenz Curve calcs (from ineq)
lorenz_uni <- Lc(uni_claims)
lorenz_psp <- Lc(df_nonzero$earnings)
lorenz_ens <- Lc(ens$tokens)
lorenz_rbn <- Lc(rbn_claims)

#make the Gini table
gini_table <- tibble(Protocol = c("Uniswap", "Ribbon Finance", "Ethereum Name Service", "Paraswap"),
                     Token = c("UNI", "RBN", "ENS", "PSP"),
                     Gini = c(Gini(uni_claims), Gini(rbn_claims), Gini(ens$tokens), Gini(df_nonzero$earnings)),
                     Theil = c(Theil(uni_claims), Theil(rbn_claims), Theil(ens$tokens), Theil(df_nonzero$earnings)),
                     Atkinson = c(Atkinson(uni_claims), Atkinson(rbn_claims), Atkinson(ens$tokens), Atkinson(df_nonzero$earnings))) %>%
                     arrange(Gini)

#qplot(x =lorenz_ens$p, y = lorenz_ens$L)
#qplot(x =lorenz_psp$p, y = lorenz_psp$L)

# Token Launches & Airdrops

Token launches happen in many different ways.  Tokens can be issued directly to team members or investors, vested in time-locked contracts for later distribution, trickled out to liquidity providers or miners as rewards for providing a service to the protocol, or delivered directly to users via airdrops.  Many protocols launch a token when (or before) a protocol launches as a means for raising capital.  Some protocols launch without a token and add one later, often using airdrops to reward prior users of the protocol for their support.  Airdrop design is tricky - it is very difficult to satisfy everyone all of the time.  This article will explore what makes an effective retroactive airdrop design, using the recent Paraswap airdrop as an example.


# The Paraswap Airdrop  
In November 2021 Paraswap, a multi-chain DEX aggregation protocol, launched the PSP token to decentralise ownership of the protocol.  7.5% of the token issuance was given to Paraswap users to reward them for their prior use of the protocol.  This act help to boostrap the Paraswap DAO and give the community the chance to contribute to and have a say over the future of Paraswap.  150m PSP tokens were issued to almost 20,000 wallets, with users receiving either 
5200, 7600 or 10400 PSP tokens, worth as much as USD15,000 in the days immediately after the airdrop.

![Paraswap Token Distribution](https://doc.paraswap.network/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-MJ6vzFu0ce0AHXkpGfO%2Fuploads%2FHbgMx3SCTT5aAQCDO2Zw%2Fimage.png?alt=media&token=5ef72958-e84c-460d-81be-f9d1261ae9b9)


# Airdrop Design Goals

To assess how effective an airdrop design is we need to have some goals against which to compare.  The goals of an airdrop can be subjective - developers, investors and other team insiders may have a view on what a good airdrop looks like, and the protocol users may have another.  There will also be a noisy group of people totally unrelated to the project who will deliver their verdict to anyone who will listen.  We will look airdrop design goals from a couple of angles to see if there is common ground

## Goals of the Paraswap Team

The Paraswap team posted a [detailed article](https://medium.com/paraswap/whats-an-active-user-clarifying-psp-token-distribution-filtering-logic-81df6096d410) on their logic behind the airdrop distribution mechanism.  This touched on a number of their goals:
- They wanted to reward "active" users
- They wanted to avoid a volume based solution, as this favours whales with large bags
- They wanted to filter out airdrop hunters - people who only use the protocol to get the airdrop and may use multiple wallets to get multiple airdrops.  Ribbon Finance was subject to this kind of attack in a [well publicised incident.](https://thedefiant.io/ribbon-finance-divergence-airdrop/)
- They wanted the airdrop quantity to be based on activity level
- They wanted to ensure geniune users only, avoiding users who were wash trading or wrapping/unwrapping tokens

The team put these goals in place in order to give tokens to community members who are more likely to be aligned with the success of the protocol.  People who are active and regular users are more likely to join the DAO, contribute and vote on the future of Paraswap.

## The Paraswap Airdrop Rules

To execute on these goals, the Paraswap team started with every wallet that had ever interacted with the project (1.3m wallets) and filtered down to 20,000 "active users".  They used the following principals when filtering:
- Ignoring wash-trading and wrap/unwrap transactions
- Removing tokens with low trading volumes (<$1m)
- Actively looked for airdrop hunters and filtering them out (addresses that had patterns of scripted or automated repeat transactions)
- Filtering out accounts with low balances or low holding durations
- Making a minimum transaction threshold of 6 transactions in the last 6 months
- Clustering accounts which transfer to each other

With the 20,000 address eligible for rewards determined, the Paraswap team decided to allocate one of 3 reward tiers - 5200, 7600 or 10400 tokens.  They used a points-based solution to determine this with these factors determining the size of the reward:
- Total transaction count
- Max value in a single transaction
- Total volume of transactions
- Paraswap used on more than one blockchain

## Community Expectations

Airdrops are an emotive issue, especially with recent airdrops being worth 5 figures (dydx, ENS) for protocol users.  There is no shortage of public commentary around what makes a good airdrop.  Much of this can be distilled into concepts of *fairness*, which seems to be based on:
- Rewarding regular or frequent users
- Punishing users who game the airdrop via multiple wallets or many small transactions
- Egalitarian distribution - spreading the rewards evenly amongst users and not allowing whales to dominate

It should be said that the airdrop doesn't have to meet the expectations of the broader community.  Prior to the airdrop, the fate of the token is in the hands of the team.  The future success of the token price & usage may, however, depend on how well received it is by future holders & DAO members.




# Measuring a Effective Airdrop

We see considerable overlap between the design goals of the Paraswap team and the concepts of fairness outlined above.  To assess how well the airdrop met our goals we will focus on measuring outcomes in three key areas.  These are:
- Rewarding frequent users
- Sybil Resistance - filtering airdrop hunters so that only genuine users are rewarded
- Equality of distribution - preventing a small number of whales getting all the tokens

To examine these outcomes, all Paraswap swap transactions were downloaded from the Ethereum and Polygon tables of [Flipside Crypto](https://flipsidecrypto.com).  Paraswap also operates on the Binance Smart Chain and Avalanche networks, however data for these chains were not available to be used in this analysis.  The total number of transactions on each network are shown in the table below.  There are over 580k unique wallets and nearly 2.5m transactions.  Note that the volume of transactions on the Polygon network is an order of magnitude bigger than Ethereum, reflecting the cheaper transaction fees.


In [20]:
#hide_input
from google.colab import data_table
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:,.0f}"
df_p = %R eth_users %>% bind_rows(poly_users) %>% bind_rows(all_users)
df_p

,Network,Users,Transactions
1,Ethereum,127380.0,331008.0
2,Polygon,465818.0,2153474.0
3,Total,584191.0,2484482.0


## Goal 1 - Rewarding Active Users
Active users have skin in the game.  They like the product and keep coming back to use it.  They have an interest in the protocol succeeding because it will be there to use in the future.  We will measure how well the Paraswap airdrop rewards regular users by looking at the percentage of users who received the airdrop against the number of transactions the user has performed.  We would expect to see groups of users with higher transaction counts be more likely to get an airdrop reward.

The following chart shows the % of users who received the airdrop vs the number of transactions on Paraswap.  This data combines both the Ethereum and Polygon transactions.  Here we see the expected results - as transaction count rises, so does the likelihood of an airdrop.  The orange line is the overall average likelihood of getting an airdrop - very low at just under 3%.  

We see that once a threshold of 20 transactions is reached, the likelihood improves considerably.  Of users with over 100 transactions, 90% received the airdrop.  We can also see the threshold value of 6 transactions in the data.  Users with less than this had a very low likelihood of getting rewards and must have satisfied the criteria on BSC or Avax chains.

In [21]:
#hide_input
#plot airdrop % by number of transaction count for both networks
df_p = %R by_tx_bucket
fig = px.bar(df_p
             , x = "tx_bucket"
             , y = "percent"
             , labels=dict(tx_bucket="No of Txs", percent="% getting airdrop")
             , title= "Percentage of Paraswap Users who got the Airdrop by Transaction Count"
             , template="simple_white", width=800, height=800/1.618
             )
fig.add_trace(go.Scatter(x=df_p["tx_bucket"]
                         , y=df_p["average"]
                         , name="Average %"
                         , mode = "lines")
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    title_text=None
))
fig.update_yaxes(title_text='% of Users who Received Airdrop')
fig.update_xaxes(title_text='Paraswap Transaction Count Ethereum and Polygon')
fig.show()

The odd shape of the chart above shows the influence of transaction value and of transactions on the other chains.  The chart below shows the same data but is restricted to Ethereum only.  As Ethereum transactions are likely to be higher in value on average (due to higher fees), users will be more likely to satisfy transaction USD volume requirements than on other chains.  Thus we get a better alignment between transaction count and likelihood of receiving an airdrop.

In [22]:
#hide_input
#plot airdrop % by number of transaction count for eth
df_p = %R eth_by_tx_bucket
fig = px.bar(df_p
             , x = "eth_tx_bucket"
             , y = "percent"
             , labels=dict(eth_tx_bucket="No of Txs", percent="% getting airdrop")
             , title= "Percentage of Paraswap Users who got the Airdrop by Transaction Count - Ethereum"
             , template="simple_white", width=800, height=800/1.618
             )
fig.add_trace(go.Scatter(x=df_p["eth_tx_bucket"]
                         , y=df_p["average"]
                         , name="Average %"
                         , mode = "lines")
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    title_text=None
))
fig.update_yaxes(title_text='% of Users who Received Airdrop')
fig.update_xaxes(title_text='Paraswap Transaction Count on Etherum')
fig.show()

## Goal 2 - Rewarding Real Users

Noone likes a cheat.  They are everywhere in the web 3 world.  Whether it's [cheating motivational speakers out of cartoon jpegs](https://twitter.com/calvinbecerra/status/1454962642339074048), [filling up NFT whitelists with multiple accounts](https://twitter.com/DogeWhale/status/1446232098810044421), [stealing funds in hacks or rugs](https://rekt.news) or [gaming an airdrop on behalf of a VC](https://twitter.com/gabagooldoteth/status/1446498569603756033?s=20), hardly a day goes by when we don't see the darker side of human nature.  

For an airdrop to be seen to be rewarding genuine users instead of airdrop hunters, we expect to see transaction volume as a determinant in airdrop rewards.  In a similar fashion to the transaction count chart, we can measure this by comparing the % of accounts receiving airdrop rewards across the total value of the transactions they performed.  There is a chart of this below, only covering transactions on the Etherum chain.

The overall average percentage of Ethereum users receiving airdrops was 5%, this is the orange line.  Users with less than USD5000 were less likely than average to receive an airdrop.  Users with large transaction volumes were more likely to get airdrops.  This is the behaviour we expected to see, and it shows that the scheme implemented by the Paraswap team was aligned with our notion of a fair airdrop.

Of interest in this chart is the users who *didn't* get the airdrop.  Of users with over 1m of transaction volume, 36.5% didn't receive an airdrop!  The Paraswap team described their rules as "harsh" - one can only hope that legitimate users didn't miss out due to overly zealous filtering.

In [23]:
#hide_input
#plot airdrop % by number of transaction Volume for Ethereum
df_p = %R eth_value_bucket
fig = px.bar(df_p
             , x = "eth_usd_bucket"
             , y = "percent"
             , labels=dict(eth_usd_bucket="Volume USD", percent="% getting airdrop")
             , title= "Percentage of Paraswap Users who got the Airdrop by Volume - Ethereum"
             , template="simple_white", width=800, height=800/1.618
             )
fig.add_trace(go.Scatter(x=df_p["eth_usd_bucket"]
                         , y=df_p["average"]
                         , name="Average %"
                         , mode = "lines")
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    title_text=None
))
fig.update_yaxes(title_text='% of Users who Received Airdrop')
fig.update_xaxes(title_text='Paraswap Transaction Volume on Ethereum USD')
fig.show()

## Goal 3 - Egalitarian Distribution

Our third goal in our fair airdrop was to see that the rewards were distributed evenly across the airdrop recipients.  Even though transaction volume is a factor in determing who is a real user, we don't want whales to vacuum up a large portion of the rewards.  

Measuring inequality is a well developed field in socioeconomic studies.  You may have heard of the Gini coefficient, which is often used to measure inequality in income or wealth distributions of populations.  The Gini coefficient is a number between 0 and 1 and is based on a Lorenz curve chart.  A Lorenze curve plots the cumulative proportion of the population on the x-axis, and the cumulative proportion of the income or wealth of that population proportion on the y-axis.  An example is here:

[![Lorenz Curve](https://vitalik.ca/images/gini/lorenz1.png)](https://vitalik.ca/general/2021/07/29/gini.html)

In the Lorenze curve chart, perfect equality is represented by the diagonal line from (0,0) to (1,1).  This line has a Gini coefficient of 0.  The further the Lorenz curve is from the diagonal, the worse the distribution equality and the higher the Gini coefficient. 

For us to examine the Paraswap airdrop using the Gini coefficient we need to have a comparison.  On its own, a Gini coefficient doesn't tell us much.  Data from three other retroactive token airdrops were collected and analysed to compare against Paraswap:
- Uniswap UNI token - 400 UNI tokens were distributed to any wallet that had swapped or provided liquidity to Uniswap - even if it was a failed transaction.  Over 250k wallets were eligible.  On top of this, LPs were airdropped tokens in proportion to the size and length of liquidity provision.
- Ribbon Finance RBN token - RBN tokens were distributed to protocol users, Ribbon discord members and to users of other options protocols.  Around 1600 wallets were eligible.
- Ethereum Name Service ENS token - [tokenised responsibility](https://twitter.com/BrantlyMillegan/status/1458167522340052992) was distributed to wallets which had registered an ENS name.  137k wallets were eligible for the airdrop.

The chart below shows the Lorenz curve for the Paraswap airdrop, compared with curves for the airdrops mentioned above.  As described earlier, perfect equality is the diagonal line.  The closer the Lorenz curve for the airdrop to the diagonal, the more equal the distribution of tokens.

Paraswap is the clear winner here.  By restricting tokens to one of only three values, and not having volume drive the quantity of tokens, they have achieved the fairest distribution of the 4 examined.  The Uniswap line was instructive here - this was the only one of the 4 airdrops to have a component which was a function of volume.  This resulted in 32% of the rewards going to 1% of receipients - the kick in the chart in the upper RHS.

![Lorenz Curves](https://github.com/scottincrypto/analytics/raw/master/data/paraswap_lorenz.png)



In [28]:
#hide_input
#plot airdrop % by number of transaction value for Ethereum
########################################
#commented out due to fastpages borking on large chart
#uncomment to run this locally
#uni_x = %R lorenz_uni$p * 100
#uni_y = %R lorenz_uni$L * 100
#psp_x = %R lorenz_psp$p * 100
#psp_y = %R lorenz_psp$L * 100
#ens_x = %R lorenz_ens$p * 100
#ens_y = %R lorenz_ens$L * 100
#rbn_x = %R lorenz_rbn$p * 100
#rbn_y = %R lorenz_rbn$L * 100
#ideal = %R seq(0,100)#

#fig = go.Figure(data=go.Scatter(x=psp_x
#                                , y=psp_y
#                                , mode="lines"
#                                , name="Paraswap"))
#fig.add_trace(go.Scatter(x=uni_x
#                                , y=uni_y
#                                , mode="lines"
#                                , name="Uniswap"))
#fig.add_trace(go.Scatter(x=ens_x
#                                , y=ens_y
#                                , mode="lines"
#                                , name="ENS"))
#fig.add_trace(go.Scatter(x=rbn_x
#                                , y=rbn_y
#                                , mode="lines"
#                                , name="Ribbon"))
#fig.add_trace(go.Scatter(x=ideal
#                                , y=ideal
#                                , mode="lines"
#                                , name="Perfect Equality"))#

#fig.update_layout(legend=dict(
##    yanchor="top",
#    y=0.99,
##    xanchor="left",
#    x=0.01,
#    title_text=None
#))
#fig.update_layout(title = "Lorenz Curves for a Selection of Airdrops")
#fig.update_layout(template="simple_white", width=800, height=800)
#fig.update_xaxes(title_text='Cumulative % of Population')
#fig.update_yaxes(title_text='Cumulative Share of Airdrop')
#fig.show()

Whilst it is easy to identify Paraswap as the most equal distribution, it's difficult to see what is next best as the curves cross.  This is where coefficients can be helpful to give a numerical assessment of inequality.  The Gini coefficient is the most common measure of this, but this can have problems as outlined in a [paper by Vitalik](https://vitalik.ca/general/2021/07/29/gini.html).  The Gini can be less useful in situations where a small proportion of people have a large amount of a resource, and everyone else has an equal share of the remainder.  This is the Distopia B scenario in Vitalik's paper and describes the Uniswap situation above.  

The table below shows the Gini coefficients for the 4 airdrops. Also included is the Thiel L Index, which is suggested by Vitalik to be more robust against Distopia B situations like Uniswap.  Also included is the Atkinson Index, a transformed version of the Thiel Index which ranges between 0 and 1.  (Thiel index is unbounded above).  

The Paraswap index has the lowest index across all three measures, confirming our assessment that this is the airdrop with the least inequality of distribution.  We can see in the table the difference in position of Uniswap across Gini and Thiel/Atkinson.  It has the 2nd most equal Gini coefficient but the most inequal Thiel/Atkinson index due to the high portion of tokens allocated to a small number of LPs.  

In [27]:
#hide_input
#output the Gini coefficient table
from google.colab import data_table
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:,.3f}"

df_p = %R gini_table
df_p

,Protocol,Token,Gini,Theil,Atkinson
1,Paraswap,PSP,0.127323,0.029257,0.014755
2,Uniswap,UNI,0.324907,1.467938,0.269249
3,Ethereum Name Service,ENS,0.402711,0.272020,0.144508
4,Ribbon Finance,RBN,0.457500,0.375472,0.168840


# Conclusions - Effective or not?

The Paraswap airdrop was a novel approach.  With airdrop hunting now a [common pursuit](https://airdrops.io/speculative/), the team took a stand and tried to allocate tokens to users who will support the project into the future, rather than mercenary actors who would dump as soon as they claimed.  They took a potential pool of 1.3m wallets and cut it down to 20,000 using an agressive filtering mechanism.  

Our data demonstrated tokens were allocated:
- Primarily to regular users
- Primarily to genuine users
- Relatively equally amongst all recipients

The Paraswap approach aligned with some common notions of fairness, as well as meeting the goals set out by the team.  Based on this, it is reasonable to conclude this was an effective airdrop.

Lessons for future airdrop teams from the Paraswap experience include:
- Set clear goals and tune the airdrop parameters to meet them.
- Identify who you want your future token holders to be?  DAO members?  Customers?  LPs?  Target the airdrop to the groups you want as your owners.
- Don't be afraid to cut hard because it's a dark forest out there.  There is a value asymmetry here - the large majority who perceive the airdrop to be fair will outweigh the unfortunate few genuine users who missed out due to stringent rules.  Noone likes to see cheaters & scammers win.
- A generous but egalitarian distribution will keep most people happy